In [8]:

!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer


In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('./data/fhv_tripdata_2021-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [20]:
y_pred

array([14.53986486, 13.74042222, 15.59333908, ..., 15.83492293,
       16.78317605, 19.65462607])

In [11]:
print('mean of the predictions = ',sum(y_pred)/len(y_pred))

mean of the predictions =  16.191691680048798


In [17]:
year=2021
month=2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
df.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'duration', 'ride_id'],
      dtype='object')

In [19]:
df['ride_id'] 

1                2021/02_1
2                2021/02_2
3                2021/02_3
4                2021/02_4
5                2021/02_5
                ...       
1037687    2021/02_1037687
1037688    2021/02_1037688
1037689    2021/02_1037689
1037690    2021/02_1037690
1037691    2021/02_1037691
Name: ride_id, Length: 990113, dtype: object

In [21]:
df_final_pq = pd.DataFrame()
df_final_pq['ride_id'] = df['ride_id']
df_final_pq['predictions'] = y_pred

In [22]:
df_final_pq.head()

,ride_id,predictions
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [23]:
#writing the results as a parquet file:
df_final_pq.to_parquet(
    './df_results.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [14]:
df.index

Int64Index([      1,       2,       3,       4,       5,       6,       7,
                  9,      10,      11,
            ...
            1037682, 1037683, 1037684, 1037685, 1037686, 1037687, 1037688,
            1037689, 1037690, 1037691],
           dtype='int64', length=990113)

In [25]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1565 bytes to starter.py
